In [ ]:
# %load_ext cudf.pandas
# import cupy as np
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from pandas.tseries.offsets import MonthEnd, DateOffset
import gc
import numba
import glob
import os
from joblib import Parallel, delayed
import os
numba.set_num_threads(8) #-----------> Change according to individual CPU thread count or potential compute power restrictions, this utilizes maximum cores
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
output_dir = 'data\\permnos'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
chunk_size = 500000
last_permno = None
temp_df = pd.DataFrame()

# Read the CSV file in chunks due to insufficient memory
for chunk in pd.read_csv('data/market_data_daily_full.csv', chunksize=chunk_size, low_memory=False):
    if last_permno is None:
        last_permno = chunk['PERMNO'].iloc[0]
    curr_permno = chunk['PERMNO'].iloc[-1]


    if curr_permno != last_permno:
        temp_df = pd.concat([temp_df, chunk[chunk['PERMNO'] == last_permno]], ignore_index=True)
        temp_df.to_pickle(f'{output_dir}/market_data_daily_{last_permno}.pkl')


        for permno, group in chunk.groupby('PERMNO'):
            if permno != last_permno and permno != curr_permno:
                group.to_pickle(f'{output_dir}/market_data_daily_{permno}.pkl')
        

        temp_df = pd.DataFrame()
        temp_df = pd.concat([temp_df, chunk[chunk['PERMNO'] == curr_permno]], ignore_index=True)
        last_permno = curr_permno
    else:
        temp_df = pd.concat([temp_df, chunk], ignore_index=True)
gc.collect()

In [ ]:
birth_data = pd.read_csv('data/birthdata.csv')
acc_data_y = pd.read_csv('data/accounting_data_yearly.csv')
acc_data_q = pd.read_csv('data/accounting_data_quarterly.csv')
ff3_data_daily = pd.read_csv('data/ff3_data_daily.csv')
sic_map = pd.read_csv("data/sic_amort.csv", names=["SIC", "Amortization_Qtrs"]).set_index("SIC")["Amortization_Qtrs"].to_dict()
gind_map = pd.read_csv("data/gind_amort.csv", names=["GIND", "Amortization_Qtrs"]).set_index("GIND")["Amortization_Qtrs"].to_dict()

In [ ]:
permnos = []
for filename in glob.glob('data\\permnos\\market_data_daily_*.pkl'):
    permno = os.path.basename(filename).split('_')[-1].split('.')[0]
    permnos.append(permno)

permnos.sort()
print(f"Number of permnos: {len(permnos)}")

In [ ]:
# Set your correct native Windows path
base_path = 'data\\permnos'

ff3_data_daily['date'] = pd.to_datetime(ff3_data_daily['date'], format='%Y%m%d')

def process_permno(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    mkt_data = pd.read_pickle(file_path)

    try:
        mkt_data['date'] = pd.to_datetime(mkt_data['date'], format='%d/%m/%Y')
    except ValueError:
        mkt_data['date'] = pd.to_datetime(mkt_data['date'], format='%Y-%m-%d')

    mkt_data = pd.merge(mkt_data, ff3_data_daily, on='date', how='inner')
    mkt_data['RET'] = pd.to_numeric(mkt_data['RET'], errors='coerce')

    first_valid_date = mkt_data[mkt_data['RET'].notna()]['date'].min()
    mkt_data = mkt_data[mkt_data['date'] >= first_valid_date].copy()

    mkt_data['excess_ret'] = mkt_data['RET'] - mkt_data['rf']
    mkt_data['logRET'] = np.log1p(mkt_data['RET'])
    mkt_data['logrf'] = np.log1p(mkt_data['rf'])
    mkt_data['logexcess_ret'] = mkt_data['logRET'] - mkt_data['logrf']

    mkt_data['quarter'] = mkt_data['date'].dt.to_period("Q")
    mkt_data['is_quarter_end'] = mkt_data.groupby(['PERMNO', 'quarter'])['date'].transform('max') == mkt_data['date']

    mkt_data['logmktrf'] = np.log1p(mkt_data['mktrf'])
    mkt_data['logsmb'] = np.log1p(mkt_data['smb'])
    mkt_data['loghml'] = np.log1p(mkt_data['hml'])
    mkt_data['logvwretd'] = np.log1p(mkt_data['vwretd'])
    # Save to same path 
    mkt_data.to_pickle(file_path)


Parallel(n_jobs=-1)(delayed(process_permno)(permno) for permno in permnos)
gc.collect()

#### View Sample Permno

In [ ]:
sample_permno = 10001  # for example
base_path = 'data\\permnos'
file_path = os.path.join(base_path, f'market_data_daily_{sample_permno}.pkl')

df = pd.read_pickle(file_path)
df.tail(5)

### FF3 Residual Momentum

In [ ]:
base_path = 'data\\permnos'

def compute_resff3_12_1(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['resff3_12_1'] = np.nan

    for idx, row in df.iterrows():
        if not row.get('is_quarter_end', False):
            continue

        current_date = row['date']
        start_date = (current_date - DateOffset(months=12)).replace(day=1)
        end_date = (current_date - DateOffset(months=1)).replace(day=1) + MonthEnd(0)

        window = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

        if window['date'].dt.to_period('M').nunique() < 11:
            continue

        X = window[['logmktrf', 'logsmb', 'loghml']].fillna(0).to_numpy()
        y = window['logexcess_ret'].fillna(0).to_numpy()

        if len(X) == 0 or len(y) == 0:
            continue

        model = LinearRegression().fit(X, y)
        y_hat = model.predict(X)
        residual = y - y_hat
        res_momentum = residual.sum()

        df.loc[idx, 'resff3_12_1'] = res_momentum

    df.to_pickle(file_path)
    return permno


results = Parallel(n_jobs=-1)(
    delayed(compute_resff3_12_1)(permno) for permno in permnos
)
gc.collect()

### -10 to -6 annualized returns; Seasonality

In [ ]:
def compute_seas_6_10an(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['seas_6_10an'] = np.nan

    for idx, row in df.iterrows():
        if not row.get('is_quarter_end', False):
            continue

        current_date = row['date']
        start_date = current_date - DateOffset(years=10)
        end_date = current_date - DateOffset(years=6)

        window = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

        if window['date'].dt.year.nunique() < 5:
            continue

        logrets = window['logRET'].fillna(0)
        log_cum_return = logrets.sum()
        log_ann_return = log_cum_return / 5

        df.loc[idx, 'seas_6_10an'] = log_ann_return

    df.to_pickle(file_path)
    return permno

results = Parallel(n_jobs=-1)(
    delayed(compute_seas_6_10an)(permno) for permno in permnos
)
gc.collect()

### -20 to -16 lagged annualized returns; Seasonality

In [ ]:
def compute_seas_20_16an(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['seas_20_16an'] = np.nan

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    logrets = df['logRET'].fillna(0).values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        start_date = current_date - DateOffset(years=20)
        end_date = current_date - DateOffset(years=16)

        mask = (dates >= np.datetime64(start_date)) & (dates <= np.datetime64(end_date))
        if pd.Series(dates[mask]).dt.year.nunique() < 5:
            continue

        log_cum_return = logrets[mask].sum()
        log_ann_return = log_cum_return / 5

        df.iat[idx, df.columns.get_loc('seas_20_16an')] = log_ann_return

    df.to_pickle(file_path)
    return permno

results = Parallel(n_jobs=-1)(
    delayed(compute_seas_20_16an)(permno) for permno in permnos
)
gc.collect()

### -10 to -6 non-annualized returns; Seasonality; Low Risk

In [ ]:
def compute_seas_6_10na(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['seas_6_10na'] = np.nan

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    logrets = df['logRET'].fillna(0).values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        start_date = current_date - DateOffset(years=10)
        end_date = current_date - DateOffset(years=6)

        mask = (dates >= np.datetime64(start_date)) & (dates <= np.datetime64(end_date))
        if pd.Series(dates[mask]).dt.year.nunique() < 5:
            continue

        log_cum_return = logrets[mask].sum()
        df.iat[idx, df.columns.get_loc('seas_6_10na')] = log_cum_return

    df.to_pickle(file_path)
    return permno
results = Parallel(n_jobs=-1)(
    delayed(compute_seas_6_10na)(permno) for permno in permnos
)
gc.collect()

### -15 to -11 non-annualized returns; Seasonality; Low Risk

In [ ]:
def compute_seas_11_15na(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['seas_11_15na'] = np.nan

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    logrets = df['logRET'].fillna(0).values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        start_date = current_date - DateOffset(years=15)
        end_date = current_date - DateOffset(years=11)

        mask = (dates >= np.datetime64(start_date)) & (dates <= np.datetime64(end_date))
        if pd.Series(dates[mask]).dt.year.nunique() < 5:
            continue

        log_cum_return = logrets[mask].sum()
        df.iat[idx, df.columns.get_loc('seas_11_15na')] = log_cum_return

    df.to_pickle(file_path)
    return permno
results = Parallel(n_jobs=-1)(
    delayed(compute_seas_11_15na)(permno) for permno in permnos
)
gc.collect()

### Zero Trading Days 252 window

In [ ]:
def compute_zero_trades_252(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['zero_trades_252'] = pd.NA

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    vol = df['VOL'].fillna(0).values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        start_date = current_date - DateOffset(years=1)

        mask = (dates >= np.datetime64(start_date)) & (dates < np.datetime64(current_date))
        if pd.Series(dates[mask]).dt.to_period('M').nunique() < 12:
            continue

        zero_days = (vol[mask] == 0).sum()
        df.iat[idx, df.columns.get_loc('zero_trades_252')] = zero_days

    df.to_pickle(file_path)
    return permno
results = Parallel(n_jobs=-1)(
    delayed(compute_zero_trades_252)(permno) for permno in permnos
)
gc.collect()

### Zero Trading Days 21 window

In [ ]:
def compute_zero_trades_21(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['zero_trades_21'] = pd.NA

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    vol = df['VOL'].fillna(0).values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        start_date = current_date - pd.DateOffset(months=1)

        mask = (dates >= np.datetime64(start_date)) & (dates < np.datetime64(current_date))

        # Require that at least 1 full month of data is present
        if pd.Series(dates[mask]).dt.to_period('M').nunique() < 1:
            continue

        zero_days = (vol[mask] == 0).sum()
        df.iat[idx, df.columns.get_loc('zero_trades_21')] = zero_days

    df.to_pickle(file_path)
    return permno

results = Parallel(n_jobs=-1)(
    delayed(compute_zero_trades_21)(permno) for permno in permnos
)
gc.collect()

### FF3 21-day Idiosyncratic Volatility 

In [ ]:
def compute_ivol_ff3_21d(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['month'] = df['date'].dt.to_period('M')
    df['ivol_ff3_21d'] = np.nan

    # precompute grouped structure once
    grouped = dict(tuple(df.groupby('month')))

    for idx, row in enumerate(df.itertuples()):
        if not row.is_quarter_end:
            continue

        current_month = row.date.to_period('M')
        monthly_data = grouped.get(current_month)

        if monthly_data is None or len(monthly_data) < 15:
            continue

        monthly_data = monthly_data.dropna(subset=['logRET', 'logrf', 'mktrf', 'smb', 'hml'])
        if len(monthly_data) < 15:
            continue

        logexcess = monthly_data['logRET'].values - monthly_data['logrf'].values
        X = monthly_data[['logmktrf', 'logsmb', 'loghml']].values

        try:
            model = LinearRegression().fit(X, logexcess)
            residuals = logexcess - model.predict(X)
            ivol = np.std(residuals)
            df.iat[idx, df.columns.get_loc('ivol_ff3_21d')] = ivol
        except Exception:
            continue

    df.to_pickle(file_path)
    return permno
results = Parallel(n_jobs=-1)(
    delayed(compute_ivol_ff3_21d)(permno) for permno in permnos
)
gc.collect()

### Firm Age

In [ ]:
print("birth_data columns:", birth_data.columns.tolist())
print(birth_data.head(3))

In [ ]:
base_path = 'data\\permnos'

birth_data = pd.read_csv("data/birthdata.csv")
birth_data.columns = birth_data.columns.str.strip().str.upper()
birth_data['BEGDAT'] = pd.to_datetime(birth_data['BEGDAT'], dayfirst=True)

def compute_firm_age(permno, birth_data):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception:
        return permno  

    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # Merge on PERMNO (not PERMCO)
    df = df.merge(birth_data[['PERMNO', 'BEGDAT']], on='PERMNO', how='left')

    if 'BEGDAT' not in df.columns:
        return permno

    days_alive = (df['date'] - df['BEGDAT']).dt.days
    df['firm_age'] = round((days_alive / 365) * 4) / 4
    df.loc[~df['is_quarter_end'], 'firm_age'] = pd.NA

    df.to_pickle(file_path)
    del df
    gc.collect()

    return permno

batch_size = 500  # You can tweak this according to processing power and memory
num_batches = (len(permnos) + batch_size - 1) // batch_size

for i in range(num_batches):
    batch = permnos[i * batch_size : (i + 1) * batch_size]
    Parallel(n_jobs=-1)(
        delayed(compute_firm_age)(permno, birth_data) for permno in batch
    )
    gc.collect()

### Standard Jegadeesh & Titman 12-1 MOM

In [ ]:
def compute_mom_12_1(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['mom_12_1'] = np.nan

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    logrets = df['logRET'].fillna(0).values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        start_date = (current_date - DateOffset(months=12)).replace(day=1)
        end_date = (current_date - DateOffset(months=1)).replace(day=1) + MonthEnd(0)

        mask = (dates >= np.datetime64(start_date)) & (dates <= np.datetime64(end_date))
        if pd.Series(dates[mask]).dt.to_period('M').nunique() < 11:
            continue

        mom = logrets[mask].sum()
        df.iat[idx, df.columns.get_loc('mom_12_1')] = mom

    df.to_pickle(file_path)
    return permno
results = Parallel(n_jobs=-1)(
    delayed(compute_mom_12_1)(permno) for permno in permnos
)
gc.collect()

### Highest 5 days of returns scaled by monthly volatility

In [ ]:
def compute_rmax5_rvol_21d(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['rmax5_rvol_21d'] = np.nan

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    logrets = df['logRET'].values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        month_start = (current_date - DateOffset(months=1)).replace(day=1)
        month_end = current_date - MonthEnd(1)

        mask = (dates >= np.datetime64(month_start)) & (dates <= np.datetime64(month_end))
        rets = pd.Series(logrets[mask]).dropna()

        if len(rets) < 5:
            continue

        top5_sum = rets.abs().nlargest(5).sum()
        vol = rets.std()

        if vol > 0:
            rmax5_rvol = top5_sum / (np.sqrt(252) * vol)
            df.iat[idx, df.columns.get_loc('rmax5_rvol_21d')] = rmax5_rvol

    df.to_pickle(file_path)
    return permno
results = Parallel(n_jobs=-1)(
    delayed(compute_rmax5_rvol_21d)(permno) for permno in permnos
)
gc.collect()

### 6-month Share Turnover

In [ ]:
def compute_turnover_126d(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['turnover_126d'] = np.nan

    dates = df['date'].values
    is_qe = df['is_quarter_end'].values
    vol = df['VOL'].values
    shrout = df['SHROUT'].values

    for idx, row in enumerate(df.itertuples()):
        if not is_qe[idx]:
            continue

        current_date = row.date
        start_date = current_date - DateOffset(months=6)

        mask = (dates >= np.datetime64(start_date)) & (dates < np.datetime64(current_date))
        if pd.Series(dates[mask]).dt.to_period("M").nunique() < 6:
            continue

        vol_sum = vol[mask].sum()

        valid_shrout = shrout[mask]
        shrout_start = valid_shrout[0] if len(valid_shrout) > 0 else np.nan

        turnover = vol_sum / shrout_start if shrout_start != 0 else np.nan
        df.iat[idx, df.columns.get_loc('turnover_126d')] = turnover

    df.to_pickle(file_path)
    return permno
results = Parallel(n_jobs=-1)(
    delayed(compute_turnover_126d)(permno) for permno in permnos
)
gc.collect()

### Market Cap

In [ ]:
def compute_market_cap(permno):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return permno

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()

    if 'PRC' not in df.columns or 'SHROUT' not in df.columns or 'is_quarter_end' not in df.columns:
        return permno

    prc = df['PRC'].abs().values  
    shrout = df['SHROUT'].values
    is_qe = df['is_quarter_end'].values


    mc = np.full(len(df), np.nan)

    # Calculate market cap only at quarter-end
    mc[is_qe] = prc[is_qe] * shrout[is_qe]

    df['mc'] = mc

    df.to_pickle(file_path)
    return permno
Parallel(n_jobs=-1)(delayed(compute_market_cap)(permno) for permno in permnos)
gc.collect()

#### View Sample Permno Again. All variables defined above should be present as columns

In [ ]:
df.head()

In [ ]:
csv_path = 'data\\market_data_daily_sample_10001.csv'
df.to_csv(csv_path, index=False)

In [ ]:
gc.collect()       

### Aggregation of data into a main market variable file

In [ ]:
from tqdm import tqdm
def aggregate_quarterly_for_permno(permno, output_dir):
    file_path = os.path.join(base_path, f'market_data_daily_{permno}.pkl')
    try:
        df = pd.read_pickle(file_path)
    except Exception as e:
        print(f"Failed to load PERMNO {permno}: {e}")
        return

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy()
    df['quarter'] = df['date'].dt.to_period('Q')
    df['is_quarter_end'] = df.groupby('quarter')['date'].transform('max') == df['date']

    if 'logRET' not in df.columns:
        return  # skip if missing core columns

    qagg = (
        df.groupby('quarter').agg({
            'logRET': 'sum',
            'logvwretd': 'sum',
            'VOL': 'sum',
            'SHROUT': 'last',
            'resff3_12_1': 'last',
            'zero_trades_252': 'last',
            'zero_trades_21': 'last',
            'turnover_126d': 'last',
            'ivol_ff3_21d': 'last',
            'rmax5_rvol_21d': 'last',
            'mom_12_1': 'last',
            'firm_age': 'last',
            'seas_6_10an': 'last',
            'seas_6_10na': 'last',
            'seas_11_15na': 'last',
            'seas_20_16an': 'last',
            'mc': 'last',
            'logmktrf': 'sum',
            'logsmb': 'sum',
            'loghml': 'sum',
            'logrf': 'sum'
        }).reset_index()
    )
    

    qagg['RET'] = np.exp(qagg['logRET']) - 1
    qagg['vwretd'] = np.exp(qagg['logvwretd']) - 1
    qagg['mktrf'] = np.exp(qagg['logmktrf']) - 1
    qagg['smb'] = np.exp(qagg['logsmb']) - 1
    qagg['hml'] = np.exp(qagg['loghml']) - 1
    qagg['rf'] = np.exp(qagg['logrf']) - 1

    # Attach PERMNO + quarter-end date
    end_dates = df[df['is_quarter_end']][['quarter', 'date']].drop_duplicates().rename(columns={'date': 'qtr_end_date'})
    qagg = qagg.merge(end_dates, on='quarter', how='left')
    qagg['PERMNO'] = permno

    output_path = os.path.join(output_dir, f'quarterly_{permno}.feather')
    qagg.to_feather(output_path)
output_dir = r'D:\Master Thesis\Code\quarterly_aggregated'
os.makedirs(output_dir, exist_ok=True)

Parallel(n_jobs=6)(
    delayed(aggregate_quarterly_for_permno)(permno, output_dir)
    for permno in tqdm(permnos, desc="Processing PERMNOs")
)

#### Aggregation of all market files into one quarterly

In [ ]:
import glob

files = glob.glob(os.path.join(output_dir, 'quarterly_*.feather'))
df_list = [pd.read_feather(f) for f in files]
full_quarterly = pd.concat(df_list, ignore_index=True)
full_quarterly.to_csv(r'D:\Master Thesis\Code\mkt_data_quarterly_full.csv', index=False)

## Fundamental Data

In [ ]:
gc.collect()

#### Backfilling historical GIND values according to their future classification first, and their SIC classification second

In [ ]:
acc_data_q['sic'] = pd.to_numeric(acc_data_q['sic'], errors='coerce').astype('Int64')
acc_data_q['gind'] = pd.to_numeric(acc_data_q['gind'], errors='coerce').astype('Int64')


#Impute from self-history

# Get most common known GIND per firm PERMNO
firm_gind_map = (
    acc_data_q[acc_data_q['gind'].notna()]
    .groupby('LPERMNO')['gind']
    .agg(lambda x: x.value_counts().index[0])
    .reset_index()
    .rename(columns={'gind': 'gind_from_self'})
)

# Merge into main data
acc_data_q = acc_data_q.merge(firm_gind_map, on='LPERMNO', how='left')

# Flag and fill
acc_data_q['gind_self_flag'] = acc_data_q['gind'].isna() & acc_data_q['gind_from_self'].notna()
acc_data_q['gind'] = acc_data_q['gind'].fillna(acc_data_q['gind_from_self'])
acc_data_q = acc_data_q.drop(columns=['gind_from_self'])


#Impute from SIC-level peers

# Build from SIC code to most common GIND code from all known GINDs
sic_to_gind_map = (
    acc_data_q[acc_data_q['gind'].notna()]
    .groupby('sic')['gind']
    .agg(lambda x: x.value_counts().index[0])
    .reset_index()
    .rename(columns={'gind': 'gind_from_sic'})
)

acc_data_q = acc_data_q.merge(sic_to_gind_map, on='sic', how='left')

acc_data_q['gind_sic_flag'] = acc_data_q['gind'].isna() & acc_data_q['gind_from_sic'].notna()
acc_data_q['gind'] = acc_data_q['gind'].fillna(acc_data_q['gind_from_sic'])

acc_data_q = acc_data_q.drop(columns=['gind_from_sic'])

In [ ]:
print("Missing GINDs:", acc_data_q['gind'].isna().sum())

#### Timeliness of 10-Q Reports

In [ ]:
acc_data_q['pdateq'] = pd.to_datetime(acc_data_q['pdateq'], errors='coerce')
acc_data_q['rdq'] = pd.to_datetime(acc_data_q['rdq'], errors='coerce')
acc_data_q['datadate'] = pd.to_datetime(acc_data_q['datadate'], errors='coerce')

pdate_ok = acc_data_q['pdateq'].notna() & (acc_data_q['pdateq'] <= acc_data_q['datadate'])
rdq_ok = acc_data_q['rdq'].notna() & (acc_data_q['rdq'] <= acc_data_q['datadate'])
update_final = acc_data_q['updq'] == 3
update_prelim = acc_data_q['updq'] == 2

total = len(acc_data_q)
timely_pdate = pdate_ok.sum()
timely_rdq_final = ((rdq_ok) & update_final).sum()
rdq_prelim = ((rdq_ok) & update_prelim).sum()
missing_rdq_pdate = ((~pdate_ok) & (~rdq_ok)).sum()

print(f"Total rows: {total:,}")
print(f"PDATEQ timely: {timely_pdate:,} ({timely_pdate / total:.2%})")
print(f"RDQ timely and final (updq=3): {timely_rdq_final:,} ({timely_rdq_final / total:.2%})")
print(f"RDQ timely but preliminary (updq=2): {rdq_prelim:,} ({rdq_prelim / total:.2%})")
print(f"Missing or delayed RDQ and PDATEQ: {missing_rdq_pdate:,} ({missing_rdq_pdate / total:.2%})")

#### Flag potentially late fillings

In [ ]:
# Boolean masks
pdate_ok = acc_data_q['pdateq'].notna() & (acc_data_q['pdateq'] <= acc_data_q['datadate'])
rdq_ok = acc_data_q['rdq'].notna() & (acc_data_q['rdq'] <= acc_data_q['datadate'])
update_final = acc_data_q['updq'] == 3
update_prelim = acc_data_q['updq'] == 2

# Delayed or missing reporting (requires shift by 2)
delayed_mask = (~pdate_ok) & (~rdq_ok | ~(update_final | update_prelim))

# Timely data (shift by 1)
timely_mask = ~delayed_mask

# Add to df for later merging with acc_data
acc_data_q['delay_flag'] = delayed_mask.astype(int)

In [ ]:
### Merge yearly and quarterly accounting data ###
acc_data_q.rename(columns={'fyearq': 'y', 'fqtr': 'qtr'}, inplace=True)
acc_data_y.rename(columns={'fyear': 'y'}, inplace=True)
acc_data = pd.merge(acc_data_q, acc_data_y[['LPERMNO', 'y', 'pstkrv', 'pstkl']], on=['LPERMNO', 'y'], how='left')

acc_data.rename(columns={'LPERMNO': 'PERMNO'}, inplace=True)

In [ ]:
### Convert year-to-date accounting data to quarterly data ###
def ytd_to_qtr(data, col):
    colq = col + '_q'
    data[colq] = data.groupby(['PERMNO', 'y'])[col].diff()
    data[colq] = data[colq].fillna(data[col])

ytd_cols = ['oancfy', 'scstkcy', 'sstky', 'wcapchy']
for col in ytd_cols:
    ytd_to_qtr(acc_data, col)

In [ ]:
# Load GICS and SIC Industry classification with their respective amortization life in quarters
acc_data['GIND'] = acc_data['gind'].fillna(0).astype(int)
acc_data['SIC'] = acc_data['sic'].fillna(0).astype(int)

def get_classification(row):
    if row['GIND'] != 0:
        return ('GIND', row['GIND'])
    else:
        return ('SIC', row['SIC'])

acc_data['industry_class'] = acc_data.apply(get_classification, axis=1)

# GIND as industry code — 0 means 'unknown', exclude those later
acc_data['industry_code'] = acc_data['GIND']
acc_data.loc[acc_data['industry_code'] == 0, 'industry_code'] = np.nan

# Convert to categorical 
acc_data['industry_cat'] = acc_data['industry_code'].astype('category')

# R&D amortization
acc_data['R&D'] = acc_data['xrdq'].fillna(0)

def rd_ammort(group: pd.DataFrame, kind: str, code: int) -> pd.DataFrame:
    if kind == 'GIND':
        rd_qtrs = gind_map.get(code, 12)
    else:
        rd_qtrs = sic_map.get(code, 12)

    group = group[['PERMNO', 'y', 'qtr', 'R&D']]
    group = group.sort_values(['PERMNO', 'y', 'qtr'])
    group['R&D_ammort'] = (
        group.groupby('PERMNO')['R&D']
        .shift(1, fill_value=0)
        .rolling(window=rd_qtrs, min_periods=1)
        .sum() / rd_qtrs
    )
    return group

# Apply amortization logic
tmp = acc_data.groupby('industry_class', group_keys=False).apply(lambda g: rd_ammort(g, *g.name))

# Merge back
acc_data = acc_data.merge(tmp[['PERMNO', 'y', 'qtr', 'R&D_ammort']], on=['PERMNO', 'y', 'qtr'], how='left')
acc_data['R&D cum'] = acc_data.groupby('PERMNO')['R&D'].cumsum()
acc_data['R&D Ammort cum'] = acc_data.groupby('PERMNO')['R&D_ammort'].cumsum()
acc_data['R&D cap'] = acc_data['R&D cum'] - acc_data['R&D Ammort cum']
acc_data['R&D cap'] = acc_data['R&D cap'].fillna(0)

In [ ]:
### Calculate accounting features ###
fin = acc_data['sic'].between(6000, 6999)  # Financial SIC codes
util = acc_data['sic'].between(4900, 4999) # Utilities SIC codes


# Income Statement Items

# Revenue
acc_data['SALES'] = acc_data['saleq'].fillna(acc_data['revtq'])  # trust 0s as valid

# Financial firms: revtq → saleq → finrevq 
acc_data.loc[fin, 'SALES'] = (
    acc_data.loc[fin, 'revtq']
    .fillna(acc_data.loc[fin, 'saleq'])
    .fillna(acc_data.loc[fin, 'finrevq'].fillna(0))
)

# Remove R&D Expense from Selling, General and Administrative Expenses and from Operating Expenses
acc_data['xsgaq'] = acc_data['xsgaq'] - acc_data['R&D'].fillna(0)
acc_data['xoprq'] = acc_data['xoprq'] - acc_data['R&D'].fillna(0)

# COGS
acc_data['COGS'] = acc_data['cogsq'].fillna(acc_data['xoprq'] - acc_data['xsgaq'])

acc_data.loc[fin, 'COGS'] = (
    acc_data.loc[fin, 'cogsq']
    .fillna(acc_data.loc[fin, 'xoprq'] - acc_data.loc[fin, 'xsgaq'])
    .fillna(acc_data.loc[fin, 'finxoprq'] - acc_data.loc[fin, 'xsgaq'])
    .fillna(acc_data.loc[fin, 'xintq'] + acc_data.loc[fin, 'xsgaq'])
    .fillna(acc_data.loc[fin, 'finxintq'] + acc_data.loc[fin, 'xsgaq']).fillna(0)  
)

                                                              
# Gross Profit
acc_data['GP'] = acc_data['SALES'] - acc_data['COGS']


# Operating Expenses
acc_data['OPEX'] = acc_data['xoprq'].fillna(acc_data['cogsq'] + acc_data['xsgaq'])

acc_data.loc[fin, 'OPEX'] = (acc_data.loc[fin, 'xoprq'].fillna(acc_data.loc[fin, 'cogsq'] + acc_data.loc[fin, 'xsgaq'])
    .fillna(acc_data.loc[fin, 'finxoprq'])  
)

# EBITDA
acc_data['EBITDA'] = acc_data['oibdpq'] + acc_data['R&D']
acc_data['EBITDA'] = acc_data['EBITDA'].fillna(acc_data['SALES'] - acc_data['OPEX'] + acc_data['R&D'])

acc_data.loc[util, 'EBITDA'] = (
    acc_data.loc[util, 'oibdpq']
    .fillna(acc_data.loc[util, 'uopiq'])
    .fillna(acc_data.loc[util, 'SALES'] - acc_data.loc[util, 'OPEX'] + acc_data.loc[util, 'R&D'])
)

# Regular reported D&A
acc_data['D&A'] = acc_data['dpq'].fillna(0)

# EBIT
acc_data['EBIT'] = acc_data['oiadpq'] + acc_data['R&D'] - acc_data['R&D_ammort']
acc_data['EBIT'] = acc_data['EBIT'].fillna(acc_data['EBITDA'] - acc_data['dpq'] - acc_data['R&D_ammort'])

# Net Income
acc_data['NI'] = acc_data['ibq']
acc_data['NI'] = acc_data['NI'].fillna(acc_data['niq'])
acc_data['NI'] = acc_data['NI'] + acc_data['R&D'] - acc_data['R&D_ammort']



# Balance Sheet Items

# Cash + ST Investments
acc_data['CASHST'] = acc_data['cheq']

# Inventory
acc_data['INVENTORY'] = acc_data['invtq']
acc_data.loc[fin, 'INVENTORY'] =  acc_data.loc[fin, 'invtq'].fillna(acc_data.loc[fin, 'finivstq'])
acc_data.loc[util, 'INVENTORY'] = acc_data.loc[util, 'invtq'].fillna(acc_data.loc[util, 'uinvq'])
                                                                     
# Current Assets
acc_data['CA'] = acc_data['actq']
acc_data['CA'] = acc_data['CA'].fillna(acc_data['rectq'] + acc_data['invtq'] + acc_data['cheq'] + acc_data['acoq'])

acc_data.loc[fin, 'CA'] = (
    acc_data.loc[fin, 'actq']
    .fillna(
        acc_data.loc[fin, 'rectq'].fillna(acc_data.loc[fin, 'finreccq']) +
        acc_data.loc[fin, 'invtq'].fillna(acc_data.loc[fin, 'finivstq']) +
        acc_data.loc[fin, 'acoq'].fillna(acc_data.loc[fin, 'finacoq']) +
        acc_data.loc[fin, 'cheq'].fillna(acc_data.loc[fin, 'finchq'])
    )
)

acc_data.loc[util, 'CA'] = (
    acc_data.loc[util, 'actq']
    .fillna(
        acc_data.loc[util, 'rectq'].fillna(acc_data.loc[util, 'urectq']) +
        acc_data.loc[util, 'invtq'].fillna(acc_data.loc[util, 'uinvq']) +
        acc_data.loc[util, 'acoq'].fillna(acc_data.loc[util, 'uacoq']) +
        acc_data.loc[util, 'cheq']
    )
)


# Current Liabilities
acc_data['CL'] = acc_data['lctq']
acc_data['CL'] = acc_data['CL'].fillna(acc_data['dlcq'] + acc_data['apq'] + acc_data['txpq'] + acc_data['lcoq'])

acc_data.loc[fin, 'CL'] = (
    acc_data.loc[fin, 'lctq']
    .fillna(
        acc_data.loc[fin, 'dlcq'].fillna(acc_data.loc[fin, 'findlcq']) +
        acc_data.loc[fin, 'apq'].fillna(acc_data.loc[fin, 'finnpq']) +
        acc_data.loc[fin, 'lcoq'].fillna(acc_data.loc[fin, 'finlcoq']) +
        acc_data.loc[fin, 'txpq']
    )
)

# Operating Current Assets
acc_data['COA'] = acc_data['CA'] - acc_data['cheq']
acc_data.loc[fin, 'COA'] = acc_data.loc[fin, 'CA']

# Operating Current Liabilities
acc_data['COL'] = acc_data['CL'] - np.maximum(acc_data['dlcq'], 0)
acc_data.loc[fin, 'COL'] = acc_data.loc[fin, 'CL']

# Total Assets
acc_data['TA'] = acc_data['atq']
acc_data['TA'] = acc_data['TA'].fillna(acc_data['seqq'] + acc_data['dlttq'] + np.maximum(acc_data['lctq'], 0))
acc_data['TA'] = acc_data['TA'] + acc_data['R&D cap']

# Net Operating Assets
acc_data['NOA'] = (acc_data['TA'] - acc_data['cheq']) - (acc_data['ltq'] - acc_data['dlcq'] - acc_data['dlttq'])

# Value of Preferred Stock 
acc_data['PSTK'] = acc_data['pstkrv']
acc_data['PSTK'] = acc_data['PSTK'].fillna(acc_data['pstkl'])
acc_data['PSTK'] = acc_data['PSTK'].fillna(acc_data['pstkq'])
acc_data['PSTK'] = acc_data['PSTK'].fillna(0)

# Adjusted Book Equity
acc_data['BE'] = (acc_data['seqq'] + acc_data['R&D cap']).fillna(acc_data['ceqq'] + acc_data['PSTK'].clip(lower=0) + acc_data['R&D cap'])

acc_data.loc[util, 'BE'] = (
    (acc_data.loc[util, 'seqq'] + acc_data.loc[util, 'R&D cap'])
    .fillna(acc_data.loc[util, 'ceqq'] + acc_data.loc[util, 'PSTK'] + acc_data.loc[util, 'R&D cap'])
    .fillna(acc_data.loc[util, 'uceqq'] + acc_data.loc[util, 'PSTK'] + acc_data.loc[util, 'R&D cap'])
)

# Total Liabilities
acc_data['LT'] = acc_data['ltq']

# Debt
acc_data['STDEBT'] = acc_data['dlcq']
acc_data['LTDEBT'] = acc_data['dlttq']

acc_data.loc[util, 'LTDEBT'] = acc_data.loc[util, 'dlttq'].fillna(
    acc_data.loc[util, 'uddq'] + acc_data.loc[util, 'udmbq'] + acc_data.loc[util, 'udoltq'] + acc_data.loc[util, 'udpcoq']
)

acc_data['TOTDEBT'] = acc_data['dlttq'] + acc_data['dlcq']

# Equity Issuance
acc_data['EQIS'] = acc_data['sstky_q']

# Minority Interest
acc_data['MI'] = np.maximum(acc_data['mibtq'], 0)

# Net Working Capital
acc_data['NWC'] =  acc_data['wcapq']
acc_data['NWC'] = acc_data['NWC'].fillna(acc_data['CA'] - acc_data['CL'])

# Current Operating Working Capital
acc_data['COWC'] = (acc_data['rectq'] + acc_data['invtq']) - acc_data['apq']
acc_data['COWC'] = acc_data['COWC'].fillna(acc_data['NWC'] - acc_data['cheq'] + acc_data['dlcq'])

# Non-Current Operating Assets
acc_data['ivaoq'] = acc_data['ivaoq'].fillna(0)
acc_data['gdwlq'] = acc_data['gdwlq'].fillna(0)
acc_data['NCOA'] = acc_data['TA'] - acc_data['CA'] - acc_data['gdwlq'] - np.maximum(acc_data['ivaoq'], 0)

# Non-Current Operating Fixed Assets
acc_data['intanq'] = acc_data['intanq'].fillna(0)
acc_data['NCOFA'] = acc_data['NCOA'] - acc_data['intanq']

# Non-Current Operating Liabilities
acc_data['NCOL'] = acc_data['ltq'] - acc_data['CL'] - acc_data['dlttq']

# Net Debt
acc_data['NETDEBT'] = acc_data['TOTDEBT'] - acc_data['cheq']

# Net Non-Current Operating Assets
acc_data['NNCOA'] = acc_data['NCOA'] - acc_data['NCOL']

# Operating Accruals
lag_nwc = acc_data.groupby(['PERMNO'])['NWC'].shift(1)
lag_nncoa = acc_data.groupby(['PERMNO'])['NNCOA'].shift(1)
acc_data['OACC'] = acc_data['NWC'] - lag_nwc + acc_data['NNCOA'] - lag_nncoa

# Operating Cash Flow
acc_data['OCF'] = acc_data['oancfy_q'] + acc_data['R&D'] - acc_data['R&D_ammort']
acc_data['OCF'] = acc_data['OCF'].fillna(acc_data['NI'] - acc_data['OACC'])
acc_data['OCF'] = acc_data['OCF'].fillna(acc_data['NI'] + acc_data['dpq'] - np.maximum(acc_data['wcapchy_q'], 0))

# Market Capitalization
acc_data['MC'] =  acc_data['mkvaltq']

In [ ]:
mkt_data = pd.read_csv(r'D:\Master Thesis\Code\mkt_data_quarterly_full.csv')
mkt_data['quarter'] = mkt_data['quarter'].astype('period[Q]')
mkt_data['y'] = mkt_data['quarter'].dt.year
mkt_data['qtr'] = mkt_data['quarter'].dt.quarter

### Overlap Between Market and Fundamental Data; Permnos starting before 1961 or after 2024, who do not have fundamental/market data are dropped

In [ ]:
# Unique PERMNOs in market data
unique_mkt_permnos = mkt_data['PERMNO'].nunique()
print(f"Unique PERMNOs in market data: {unique_mkt_permnos}")

# Unique PERMNOs in accounting data
unique_acc_permnos = acc_data['PERMNO'].nunique()
print(f"Unique PERMNOs in accounting data: {unique_acc_permnos}")

In [ ]:
# Market data year range
mkt_min_year = mkt_data['y'].min()
mkt_max_year = mkt_data['y'].max()
print(f"Market data covers years: {mkt_min_year} – {mkt_max_year}")

# Accounting data year range
acc_min_year = acc_data['y'].min()
acc_max_year = acc_data['y'].max()
print(f"Accounting data covers years: {acc_min_year} – {acc_max_year}")

In [ ]:
# Step 1: Define columns to keep
accounting_fields = [
    'SALES', 'COGS', 'GP', 'OPEX', 'EBITDA', 'D&A', 'EBIT', 'NI',
    'INVENTORY', 'CA', 'CL', 'COA', 'COL', 'TA', 'NOA', 'PSTK', 'BE', 'LT',
    'STDEBT', 'LTDEBT', 'EQIS', 'MI', 'NWC', 'NCOA', 'COWC', 'CASHST',
    'NCOFA', 'NCOL', 'NETDEBT', 'NNCOA', 'OACC', 'OCF'
]

market_fields = [
    'resff3_12_1', 'mom_12_1',
    'zero_trades_252', 'zero_trades_21', 'turnover_126d', 'ivol_ff3_21d',
    'rmax5_rvol_21d', 'firm_age', 'seas_6_10an', 'seas_6_10na',
    'seas_11_15na', 'seas_20_16an'
]

# Step 2: Subset and keep MC separately
keep_cols_mkt = ['PERMNO', 'y', 'qtr', 'RET', 'vwretd', 'mc'] + market_fields
keep_cols_acc = ['PERMNO', 'y', 'qtr', 'GIND', 'SIC', 'MC'] + accounting_fields

mkt_data = mkt_data[keep_cols_mkt]
acc_data = acc_data[keep_cols_acc]

# Step 3: Merge
data_merged = pd.merge(
    mkt_data,
    acc_data,
    on=['PERMNO', 'y', 'qtr'],
    how='inner'
)

# Step 4: Fallback MC
data_merged['MC'] = data_merged['MC'].fillna(data_merged['mc'] / 1000)
data_merged.drop(columns=['mc'], inplace=True)

# Step 5: Redefine accounting_fields now that MC is finalized
accounting_fields += ['MC']

# Merge delay_flag before shifting
delay_flags = acc_data_q[['LPERMNO', 'y', 'qtr', 'delay_flag']].rename(columns={'LPERMNO': 'PERMNO'})
data_merged = pd.merge(data_merged, delay_flags, on=['PERMNO', 'y', 'qtr'], how='left')
data_merged['delay_flag'] = data_merged['delay_flag'].fillna(1).astype(int)  # assume delay if unknown

# Step 6: Fluid Shift of accounting fields conditionally (delay_flag controls 1 or 2 lags)
for col in accounting_fields:
    grouped = data_merged.groupby('PERMNO')[col]
    lag1 = grouped.shift(1)
    lag2 = grouped.shift(2)
    data_merged[col + '_shifted'] = np.where(data_merged['delay_flag'] == 1, lag2, lag1)


# Shift market fields uniformly by 1
for col in market_fields:
    data_merged[col + '_shifted'] = data_merged.groupby('PERMNO')[col].shift(1)

data_merged.drop(columns=accounting_fields + market_fields, inplace=True)
data_merged.rename(columns={c + '_shifted': c for c in accounting_fields + market_fields}, inplace=True)


# Step 7: Final formatting
fields = ['PERMNO', 'y', 'qtr', 'GIND', 'SIC', 'RET', 'vwretd'] + accounting_fields + market_fields
data_merged = data_merged[fields].sort_values(['PERMNO', 'y', 'qtr'])

# Export
data_merged.to_csv('data/data_merged_fluidshift.csv', index=False)

In [ ]:
print(f"Final merged shape: {data_merged.shape}")
print(f"Unique firms: {data_merged['PERMNO'].nunique()}")
print(f"Years covered: {data_merged['y'].min()} to {data_merged['y'].max()}")

### Shift Sanity Check; Market Data Shifted by 1 row, Accounting Data Shifted by 2 rows

In [ ]:
# Choose a common PERMNO
permno = data_merged['PERMNO'].dropna().unique()[0]

# Sort and subset raw datasets
acc_sample = acc_data[acc_data['PERMNO'] == permno].sort_values(['y', 'qtr'])[['y', 'qtr', 'SALES']]
mkt_sample = mkt_data[mkt_data['PERMNO'] == permno].sort_values(['y', 'qtr'])[
    ['y', 'qtr', 'RET', 'vwretd', 'ivol_ff3_21d', 'zero_trades_21']
]
merged_sample = data_merged[data_merged['PERMNO'] == permno].sort_values(['y', 'qtr'])[
    ['y', 'qtr', 'SALES', 'RET', 'vwretd', 'ivol_ff3_21d', 'zero_trades_21']
]

# Rename for clarity
acc_sample = acc_sample.rename(columns={'SALES': 'SALES_raw'})
mkt_sample = mkt_sample.rename(columns={
    'RET': 'RET_raw', 
    'vwretd': 'vwretd_raw',
    'ivol_ff3_21d': 'ivol_ff3_21d_raw',
    'zero_trades_21': 'zero_trades_21_raw'
})
merged_sample = merged_sample.rename(columns={
    'SALES': 'SALES_shifted',
    'RET': 'RET_current',
    'vwretd': 'vwretd_current',
    'ivol_ff3_21d': 'ivol_ff3_21d_shifted',
    'zero_trades_21': 'zero_trades_21_shifted'
})

# Merge all for comparison
check = pd.merge(merged_sample, acc_sample, on=['y', 'qtr'], how='left')
check = pd.merge(check, mkt_sample, on=['y', 'qtr'], how='left')

# Show results
pd.set_option('display.max_columns', None)
print(check.head(10).to_string(index=False))